



Built using guidance from https://arxiv.org/pdf/1512.09300.pdf

Features:
  * Uses ELU activations
  * Deconvolution uses upscaling unpool layer before affine operator, rather than spacing with zeros
  * Batch normalization after each transformation
  * Dropout layer after activation
  * abs-sum image loss rather than cross-entropy loss

# Summary

Best result so far is 10 epochs of the first training batch, with the prior and the prediction weighted equally.

Running another 10 epochs doesn't hurt the similarity results, but it does make the reconstructions worse.

Not quite as good after 10 epochs with regularization=0.1 (down-weighted prior)

In [4]:
import tensorflow as tf
 
print("running TensorFlow version {}".format(tf.__version__))

running TensorFlow version 1.8.0


In [5]:
# Control memory usage

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
# Report OOM details

run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)


# Import data

In [7]:
import os
import re

from operator import itemgetter

In [8]:
LOG_ROOT = '../../data/'

RUN_NAME = 'vae/vae_005'
SUMMARY_DIR = os.path.join(LOG_ROOT, 'logs', RUN_NAME)
MODEL_DIR = os.path.join(LOG_ROOT, 'models', RUN_NAME)
MODEL_GRAPH = os.path.join(MODEL_DIR, 'vae.meta')
MODEL_PREFIX = os.path.join(MODEL_DIR, 'vae')

DATA_SIZE = 'all_packs'
EXPT_NAME = 'expt_005'
VIS_NAME = 'vis_001'
DATA_ROOT = '/var/data/processed'

EXPT_DIR = os.path.join(DATA_ROOT, EXPT_NAME, 'data')
TFRECORDS_DIR = os.path.join('/var/data/original/tfrecords/', DATA_SIZE)
EMBEDDINGS_DIR = os.path.join(EXPT_DIR, 'embeddings')
ENCODED_IMAGES_DIR =  os.path.join(EXPT_DIR, 'images')

TRAIN_IMAGES = '/var/data/original/data/images_1pack/'

In [9]:
def _load_image(path, size):
    ''' load grayscale image from file; resize if necessary
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        raise ValueError("empty image: {}".format(path))

    if size:
        img = cv2.resize(img, (size,size), interpolation=cv2.INTER_CUBIC)

    img = img.astype(np.float32)
    return img

In [10]:
import glob
import cv2

data = []
filenames = []
for i,image_path in enumerate(glob.glob(os.path.join(TRAIN_IMAGES,'*'))):
    filenames.append(image_path[-16:-4])
    data.append(_load_image(image_path, 256))
    if(i==100):
        break

In [11]:
train_256_pattern = re.compile('^train_(?P<block_id>[0-9]{3}).tfrecords')
validate_256_pattern = re.compile('^validate_(?P<block_id>[0-9]{3}).tfrecords')
test_256_pattern = re.compile('^test_(?P<block_id>[0-9]{3}).tfrecords')
golden_256_pattern = re.compile('golden_(?P<block_id>[0-9]{3}).tfrecords')


In [12]:

ALL_TFRECORDS = os.listdir(TFRECORDS_DIR)
def get_sorted_records(pattern, directory):
     return [ \
         os.path.join(directory, _file) \
         for _file in \
         sorted([_m[0] for _m in \
             [pattern.match(_f) for _f in os.listdir(directory)] if _m]) \
     ]

GOLDEN_TFRECORDS = get_sorted_records(golden_256_pattern, TFRECORDS_DIR)
TRAIN_TFRECORDS = get_sorted_records(train_256_pattern, TFRECORDS_DIR)
VALIDATE_TFRECORDS = get_sorted_records(validate_256_pattern, TFRECORDS_DIR)
TEST_TFRECORDS = get_sorted_records(test_256_pattern, TFRECORDS_DIR)

In [13]:


TRAIN_TFRECORDS, VALIDATE_TFRECORDS,GOLDEN_TFRECORDS, TEST_TFRECORDS

(['/var/data/original/tfrecords/all_packs/train_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/validate_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/golden_003.tfrecords'],
 ['/var/data/original/tfrecords/all_packs/test_003.tfrecords'])

In [14]:
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

In [15]:
def _augment(filename, image):
    '''Placeholder for data augmentation
    '''
    image = tf.reshape(image, [256, 256, 1])
    return filename, image


def _normalize(filename, image):
    '''Convert `image` from [0, 255] -> [-0.5, 0.5] floats
    '''
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return filename, image

In [16]:
def inputs(filenames, images, batch_size, num_epochs):
    ''' Reads input data num_epochs times or forever if num_epochs is None
        returns dataset, iterator pair
    '''

    with tf.name_scope('input'):
        # TFRecordDataset opens a binary file and reads one record at a time.
        # `filename` could also be a list of filenames, which will be read in order.
       
        def gen():
            for i,_ in enumerate(filenames):
                yield(filenames[i], images[i])
                
        dataset = tf.data.Dataset.from_generator(gen,
                                                 (tf.string, tf.float32),
                                                 (tf.TensorShape([]), tf.TensorShape([256,256])))

        '''def generator():
            for i,f in enumerate(filenames):
                yield tuple(f, images[i])
            
            
            
        #dataset = tf.data.Dataset.from_generator(lambda: images, tf.float32, tf.TensorShape([256,256]))
        dataset = tf.data.Dataset.from_generator(generator, (tf.string, tf.float32) 
                                                 (tf.TensorShape([None]), tf.TensorShape([256,256])))'''
        
        # The map transformation takes a function and applies it to every element
        # of the dataset.
        
        #dataset = dataset.map(_decode)
        #dataset = dataset.shard(num_shards, shard_index)
        #dataset = dataset.filter(_filter)
        dataset = dataset.map(_augment)
        dataset = dataset.map(_normalize)

        # The shuffle transformation uses a finite-sized buffer to shuffle elements
        # in memory. The parameter is the number of elements in the buffer. For
        # completely uniform shuffling, set the parameter to be the same as the
        # number of elements in the dataset.
        dataset = dataset.shuffle(1000 + 3 * batch_size)

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size)

        iterator = dataset.make_one_shot_iterator()

    return dataset, iterator

In [84]:
# training parameters
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = 1

DROPOUT = 0.7
REGULARIZATION = 0.1

DISPLAY_EVERY = 100

## Loggers

In [85]:
def variable_summary(x, name):
    with tf.variable_scope(name):
        mean = tf.reduce_mean(x)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(x - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(x))
        tf.summary.scalar('min', tf.reduce_min(x))
        tf.summary.histogram('histogram', x)

## Component layers

In [86]:
# unpool operation doesn't yet exist in TF

def unpool_op(x, stride, name='unpool'):

    with tf.name_scope(name) as scope:

        if stride==1:
            return x

        shape = x.get_shape().as_list()
        dim = len(shape[1:-1])
        out = (tf.reshape(x, [-1] + shape[-dim:]))
        for i in range(dim, 0, -1):
            out = tf.concat([out]*stride, i)
        out_size = [-1] + [s * stride for s in shape[1:-1]] + [shape[-1]]
        out = tf.reshape(out, out_size, name=scope)
    return out

In [87]:
def convolution_layer(x, dims, stride, train, bias=None, name='conv', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())

        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        if bias is None:
             conv = tf.nn.conv2d(x, weights, strides=[1, stride, stride, 1], padding='SAME', name='conv')
        else:
             conv = tf.nn.bias_add(tf.nn.conv2d(x, weights, strides=[1, stride, stride, 1],
                        padding='SAME'), biases, name='conv')
        normalized = tf.layers.batch_normalization(conv, axis=3, training=train,
                    name='spatial_batch_norm')        
        activations = activation(normalized, name='activation')
        activations = tf.layers.dropout(activations, rate=DROPOUT, training=train, name='dropout')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', normalized)
        tf.summary.histogram('activations', activations)

        return activations

In [88]:
def deconvolution_layer(x, dims, stride, train, bias=None, name='deconv', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())
        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        unpool = unpool_op(x, stride, name='unpool')
        if bias is None:
            deconv = tf.nn.conv2d(unpool, weights, strides=[1, 1, 1, 1], padding='SAME', name='deconv')
        else:
            deconv = tf.nn.bias_add(tf.nn.conv2d(unpool, weights, strides=[1, 1, 1, 1],
                        padding='SAME'), biases, name='deconv')
        normalized = tf.layers.batch_normalization(deconv, axis=3, training=train,
                    name='spatial_batch_norm')        
        activations = activation(deconv, name='activation')
        activations = tf.layers.dropout(activations, rate=DROPOUT, training=train, name='dropout')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', deconv)
        tf.summary.histogram('activations', activations)

        return activations

In [89]:
def dense_layer(x, dims, train, bias=None, name='fc', activation=tf.nn.elu):

    with tf.variable_scope(name):

        # Parameters
        weights = tf.get_variable('w', dims,
                    initializer=tf.contrib.layers.xavier_initializer())
        if bias is not None:
            biases = tf.get_variable('b', [dims[-1]],
                        initializer=tf.random_normal_initializer())

        # Layer structure
        if bias is None:
            dense = tf.matmul(x, weights, name='dense')
        else:
            dense = tf.nn.bias_add(tf.matmul(x, weights), biases, name='dense')

        normalized = tf.layers.batch_normalization(dense, axis=1, training=train,
                    name='batch_norm')
        activations = activation(normalized, name='activation')

        # Variable summaries
        variable_summary(weights, 'weights')
        if bias is not None:
            variable_summary(biases, 'biases')
        tf.summary.histogram('pre-activations', normalized)
        tf.summary.histogram('activations', activations)

        return activations

In [90]:
def gaussian_sample(mean, stddev, name):

    with tf.variable_scope(name):

        # mean is unconstrained; stddev must be strictly positive
        stddev = 1e-6 + tf.nn.softplus(stddev)

        # actually sample
        z = mean + stddev * tf.random_normal(tf.shape(mean), 0, 1, dtype=tf.float32)

        # Variable summaries
        tf.summary.histogram('mean', mean)
        tf.summary.histogram('stddev', stddev)
        tf.summary.histogram('z', z)

        return mean, stddev, z

## Loss

In [91]:
def evaluate(x, xhat, mu, sigma):

    with tf.variable_scope('loss'):
        # Structure
        pred = tf.losses.absolute_difference(x, xhat,
                reduction=tf.losses.Reduction.MEAN)

        # offsetx = x + 0.5
        # pred = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        #         labels=offsetx, logits=xhat))

        KLdiv = 0.5 * tf.reduce_mean(tf.square(mu) + \
                    tf.square(sigma) - tf.log(1e-8 + tf.square(sigma)) - 1)

        loss = tf.add(REGULARIZATION * KLdiv, pred)

        # Summaries
        tf.summary.scalar('prediction', pred)
        tf.summary.scalar('prior', KLdiv)
        tf.summary.scalar('loss', loss)

    return loss, pred, KLdiv

## Autoencoder

In [92]:
def encoder(img, train):

    with tf.variable_scope('encoder'):

        # convolution
        conv1 = convolution_layer(img, [5, 5, 1, 64], 2, train, name='conv1')
        conv2 = convolution_layer(conv1, [5, 5, 64, 128], 2, train, name='conv2')
        conv3 = convolution_layer(conv2, [5, 5, 128, 256], 2, train, name='conv3')

        # transition
        conv3 = tf.reshape(conv3, [-1, 32*32*256], name='reshape1')

        # dense output
        fc1 = dense_layer(conv3, [32*32*256, 64], train,
                activation=tf.identity, name='fc1')

        # sample
        mu, sigma, z = gaussian_sample(fc1[:, :32], fc1[:, 32:], name='output')

    return mu, sigma, z

In [93]:
def decoder(z, train):

    with tf.variable_scope('decoder'):

        # dense input
        fc1 = dense_layer(z, [32, 32*32*256], train, name='fc1')

        # transition
        fc1 = tf.reshape(fc1, [-1, 32, 32, 256], name='reshape1')

        # deconvolution
        deconv1 = deconvolution_layer(fc1, [5, 5, 256, 128], 2, train,
                            name='deconv1')
        deconv2 = deconvolution_layer(deconv1, [5, 5, 128, 64], 2, train, 
                            name='deconv2')
        deconv3 = deconvolution_layer(deconv2, [5, 5, 64, 32], 2, train,
                            name='deconv3')
        logits = deconvolution_layer(deconv3, [5, 5, 32, 1], 1, train,
                            activation=tf.identity, name='logits')

        # put into image range for display
        with tf.name_scope('range'):
            xhat = 0.5 * tf.nn.tanh(logits)

    return xhat, logits

In [94]:
def decoder_alternative(z, train):

    with tf.variable_scope('decoder'):

        # dense input
        fc1 = dense_layer(z, [32, 32*32*256], train, name='fc1')

        # transition
        fc1 = tf.reshape(fc1, [-1, 32, 32, 256], name='reshape1')

        # deconvolution
        deconv1 = deconvolution_layer(fc1, [5, 5, 256, 128], 2, train,
                            name='deconv1')
        deconv1a = deconvolution_layer(deconv1, [5, 5, 128, 128], 2, train,
                            name='deconv1a')
        deconv2 = deconvolution_layer(deconv1a, [5, 5, 128, 64], 2, train, 
                            name='deconv2')
        deconv3 = deconvolution_layer(deconv2, [5, 5, 64, 32], 2, train,
                            name='deconv3')
        logits = deconvolution_layer(deconv3, [5, 5, 32, 1], 1, train,
                            activation=tf.identity, name='logits')

        # put into image range for display
        with tf.name_scope('range'):
            xhat = 0.5 * tf.nn.tanh(logits)

    return xhat, logits

# Build and run

## Initialize

In [111]:
import sys
import os
import argparse
from datetime import datetime

import logging
logging.basicConfig(
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    format="%(asctime)s [train/initialize] %(levelname)-8s %(message)s",
    level=logging.INFO
)

NOW_STR = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S%z")
RUN_DESC = "cross-entropy loss, 256x256 images, no bias"
RANDOM_SEED = 42


In [113]:
def initialize():

    logging.info("initializing run: {}".format(RUN_NAME))

    # write a note regarding this run
    os.makedirs(SUMMARY_DIR, exist_ok=True)
    with open(os.path.join(SUMMARY_DIR, "description.txt"), 'w') as fh:
        fh.write(NOW_STR+" "+RUN_DESC)

    # Control memory usage
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # Report OOM details
    run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)

    # Build graph
    with tf.Graph().as_default() as graph:

        # Repeatable results
        tf.set_random_seed(RANDOM_SEED)

        # Get Data
        train_dataset, train_iterator = inputs(filenames=filenames, images=data,
                batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS)
        # Data placeholder
        data_handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            data_handle, train_dataset.output_types, train_dataset.output_shapes)
        
        filename, image = iterator.get_next()
        train = tf.placeholder(tf.bool)

        # Global counter
        global_step = tf.train.get_or_create_global_step(graph)

        # Dropout
        dropout = tf.placeholder(tf.float32)

        # Autoencoder
        mu, sigma, z = encoder(image, train)
        xhat, logits = decoder(z, train)
        loss, recon, reg = evaluate(image, xhat, mu, sigma)

        # Training branch - control dependencies so batchnorm params are updated
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)\
                .minimize(loss, global_step=global_step, name='optimizer')

        # Log output for Tensorboard
        merged = tf.summary.merge_all()
        train_summary_logger = tf.summary.FileWriter(SUMMARY_DIR+'/train',
                        graph=graph, flush_secs=30)

        # Initializer
        init_variables = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

        # Save state
        tf.add_to_collection('optimizer', optimizer)

        tf.add_to_collection('filename', filename)
        tf.add_to_collection('image', image)

        tf.add_to_collection('mu', mu)
        
        tf.add_to_collection('sigma', sigma)
        tf.add_to_collection('xhat', xhat)

        tf.add_to_collection('loss', loss)
        tf.add_to_collection('recon', recon)
        tf.add_to_collection('reg', reg)

        tf.add_to_collection('data_handle', data_handle)
        tf.add_to_collection('train', train)

        tf.add_to_collection('merged', merged)

        writer = tf.train.Saver()

        # Run one step: this initializes the graph and saves our starting statistics
        with tf.Session(config=config) as session:

            session.run(init_variables)

            train_handle = session.run(train_iterator.string_handle())

            # Output header
            logging.info("  step      loss      recon     reg")

            _, step = session.run([optimizer, global_step],
                    feed_dict = { data_handle: train_handle, train: 1 },
                    options = run_options)

            loss_, recon_, reg_, summary = \
                session.run([loss, recon, reg, merged],
                           feed_dict = { data_handle: train_handle, train: 0 })
            train_summary_logger.add_summary(summary, step)

            logging.info("{: 6d} {:9.3g} {:9.3g} {:9.3g}".format(step, loss_, recon_, reg_))

            # Save graph
            logging.info("saving graph")
            writer.save(session, MODEL_PREFIX, global_step=step, write_meta_graph=False)
            writer.export_meta_graph(MODEL_GRAPH)
            


def main():

    parser = argparse.ArgumentParser(description='initialize training graph')
    initialize()
    sys.exit(0)


if __name__ == '__main__':
    main()

2018-07-06T22:19:57+0000 [train/initialize] INFO     initializing run: vae/vae_005
2018-07-06T22:20:00+0000 [train/initialize] INFO       step      loss      recon     reg
2018-07-06T22:20:06+0000 [train/initialize] INFO          0     0.222     0.211     0.108
2018-07-06T22:20:06+0000 [train/initialize] INFO     saving graph


SystemExit: 0

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Train

In [114]:
with tf.Graph().as_default() as graph:
    
    # Repeatable results
    tf.set_random_seed(0)

    # Get Data
    _, train_iterator = inputs(filenames, images=data, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS)
        
    # Log output for Tensorboard
    train_summary_logger = tf.summary.FileWriter(SUMMARY_DIR+'/train', flush_secs=30)

    # Run
    with tf.Session(config=config) as session:

        # restore
        reader = tf.train.import_meta_graph(MODEL_GRAPH)
        reader.restore(session, tf.train.latest_checkpoint(MODEL_DIR))

        # must be called after reader so that the graph is populated
        writer = tf.train.Saver()

        # get references to graph endpoints
        global_step = tf.train.get_global_step(graph)

        optimizer = tf.get_collection('optimizer')[0]
        loss = tf.get_collection('loss')[0]
        recon = tf.get_collection('recon')[0]
        reg = tf.get_collection('reg')[0]

        data_handle = tf.get_collection('data_handle')[0]
        train = tf.get_collection('train')[0]

        merged = tf.get_collection('merged')[0]

        train_handle = session.run(train_iterator.string_handle())

        # Output header
        logging.info("  step      loss      recon     reg")

        while True:
            try:
                _, step = session.run([optimizer, global_step], 
                            feed_dict = { data_handle: train_handle, train: 1 })

                if not step%DISPLAY_EVERY:

                    loss_, recon_, reg_, summary = session.run([loss, recon, reg, merged],
                            feed_dict = { data_handle: train_handle, train: 0 })
                    train_summary_logger.add_summary(summary, step)

                    logging.info("{: 6d} {:9.3g} {:9.3g} {:9.3g}".format(
                        step, loss_, recon_, reg_))

                    writer.save(session, MODEL_PREFIX, global_step=step, write_meta_graph=False)

            except tf.errors.OutOfRangeError:
                print("done")
                break

INFO:tensorflow:Restoring parameters from ../../data/models/vae/vae_005/vae-0


2018-07-06T22:20:11+0000 [train/initialize] INFO     Restoring parameters from ../../data/models/vae/vae_005/vae-0
2018-07-06T22:20:12+0000 [train/initialize] INFO       step      loss      recon     reg


done


# Test

Try out the autoencoder by running it on some test set samples.

For a collection of test images:

1. Create (image, label, mu, sigma) tuples
1. For a seed image, compute the 10 nearest images using (mu, sigma)
1. View the nearby images and their labels, comparing them to the seed image

If the VAE has worked as expected, we should find that the nearby images match the seed visually, and perhaps even match according to their labels.

## Generate document vectors

In [17]:


def gen_documents(filenames, images, RUN_NAME, BATCH_SIZE):
    
    SUMMARY_DIR = os.path.join(LOG_ROOT, 'logs', RUN_NAME)
    dir_name = SUMMARY_DIR + '/train'
    MODEL_DIR = os.path.join(LOG_ROOT, 'models', RUN_NAME)
    MODEL_GRAPH = os.path.join(MODEL_DIR, 'vae.meta')
    
    
    # Store the documents
    documents = []
    def extend(docs, m, s, xh, f, i):
        start_id = len(docs)
        docs.extend([
            {
                'filename':f_.decode('ascii') ,
                'image': i_.reshape(256, 256)+0.5,
                'sigma': s_,
                'mu': m_,
                'xhat': y_.reshape(256, 256)+0.5
            }
            for k, (m_, s_, y_, f_, i_) in enumerate(zip(m, s, xh, f, i))
        ])
        return docs

    with tf.Graph().as_default() as graph:

        # Repeatable results
        tf.set_random_seed(0)

        # Get Data
        dataset, iterator = inputs(filenames = filenames, images = images, batch_size=BATCH_SIZE, num_epochs=1)
        # Log output for Tensorboard
        summary_logger = tf.summary.FileWriter(dir_name, flush_secs=30)

        # Run
        with tf.Session(config=config) as session:

            # restore
            reader = tf.train.import_meta_graph(MODEL_GRAPH)
            reader.restore(session, tf.train.latest_checkpoint(MODEL_DIR))

            # get references to graph endpoints
            filename = tf.get_collection('filename')[0]
            image = tf.get_collection('image')[0]

            mu = tf.get_collection('mu')[0]
            sigma = tf.get_collection('sigma')[0]
            xhat = tf.get_collection('xhat')[0]

            merged = tf.get_collection('merged')[0]

            data_handle = tf.get_collection('data_handle')[0]
            train = tf.get_collection('train')[0]
            handle = session.run(iterator.string_handle())
            
            step = 0
            while True:
                try:
                    mu_, sigma_, xhat_, filename_,  image_, summary = \
                        session.run([mu, sigma, xhat, filename, image, merged],
                                feed_dict = { data_handle: handle, train: 0 })
                    documents = extend(documents, mu_, sigma_, xhat_,filename_, image_)
                    step += 1
                    summary_logger.add_summary(summary, step)                        
                    print("{}.".format(step), end="", flush=True)
        
                except tf.errors.OutOfRangeError:
                    print(".done")
                    break
    return documents

In [18]:
RUN_NAME = 'vae/vae_005'
BATCH_SIZE = 32
documents = gen_documents(filenames, data, RUN_NAME, BATCH_SIZE)

INFO:tensorflow:Restoring parameters from ../../data/models/vae/vae_005/vae-0
1.2.3.4..done


In [23]:

    
print(len(documents))

101


In [ ]:
SUBSET = "train"

In [ ]:
if SUBSET == "golden":
    golden_documents = gen_documents(GOLDEN_TFRECORDS, '/golden', 1)
elif SUBSET == "validate":
    validate_documents = gen_documents(VALIDATE_TFRECORDS, '/validate')
elif SUBSET == "test":
    test_documents = gen_documents(TEST_TFRECORDS, '/test')
elif SUBSET == "train":
    train_documents = gen_documents(TRAIN_TFRECORDS, '/train', 4)


## Generate Json

In [ ]:
import json
import scipy.misc as misc

In [ ]:
def gen_json(documents):
    
    for doc in documents:
        json_item = {
            'id':doc['id_'],
            'filename': doc['filename'],
            'subset': SUBSET,
            'encodedname': doc['filename'],
            'labels': doc['labels'].tolist(),
            'age': int(doc['age']),
            'gender': doc['gender'],
            'view': doc['view'],
            'mu':doc['mu'].tolist(),
            'sigma':doc['sigma'].tolist()
        }

        JSON_DIR = os.path.join(EMBEDDINGS_DIR, doc['filename'][:12] + ".json")
        with open(JSON_DIR, 'w') as outfile:
            json.dump(json_item, outfile, separators=(',', ':'), indent = 2)
    

In [ ]:
if SUBSET == "golden":
    gen_json(golden_documents)

elif SUBSET == "validate":
    gen_json(validate_documents)
    
elif SUBSET == "test":
    gen_json(test_documents)
    
elif SUBSET == "train":
    gen_json(train_documents)

## save images

In [ ]:

#save the encoded 256x256 vectors as png file
def save_images(documents):
    for doc in documents:

        path = os.path.join(ENCODED_IMAGES_DIR, doc['filename'])
        misc.imsave(path, doc['xhat'])


In [ ]:
if SUBSET == "golden":
    save_images(golden_documents)

elif SUBSET == "validate":
    save_images(validate_documents)
    
elif SUBSET == "test":
    save_images(test_documents)
    
elif SUBSET == "train":
    save_images(train_documents)

In [ ]:
for i,x in enumerate(train_documents):
    f = 0
print(i)